# Modelo de Machine Learning na Nuvem AWS para escolas

In [2]:
# Imports
import sklearn
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

## Dados

In [84]:
dados = pd.read_csv("data/dataset.csv")
dados.head()

,nota_exame_ingles,valor_qi,nota_exame_psicotecnico,admitido
0,5.41,115,54,0
1,7.36,105,77,1
2,7.76,96,62,1
3,5.51,112,51,0
4,7.19,97,47,1


## Pré-processamento

In [85]:
dados.shape

(300, 4)

In [86]:
# VAriáveis preditoras
X = dados.drop(columns='admitido')
y = dados['admitido']

In [87]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y , test_size=0.2, random_state=2)

In [88]:
X_treino.describe().round(0)

,nota_exame_ingles,valor_qi,nota_exame_psicotecnico
count,240.0,240.0,240.0
mean,7.0,103.0,60.0
std,1.0,8.0,15.0
min,5.0,83.0,23.0
25%,6.0,96.0,51.0
50%,7.0,104.0,60.0
75%,7.0,109.0,70.0
max,9.0,121.0,100.0


In [89]:
# Padronização dos dados
scaler = StandardScaler()

In [90]:
X_treino_scaled = scaler.fit_transform(X_treino)
X_teste_scaled = scaler.fit_transform(X_teste)

In [91]:
pd.DataFrame(X_treino_scaled).describe().round(0)

,0,1,2
count,240.0,240.0,240.0
mean,-0.0,0.0,-0.0
std,1.0,1.0,1.0
min,-2.0,-2.0,-2.0
25%,-1.0,-1.0,-1.0
50%,0.0,0.0,-0.0
75%,1.0,1.0,1.0
max,3.0,2.0,3.0


## Versão 1: Regressão Logística

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
modelo_logistico = LogisticRegression()
modelo_logistico.fit(X_treino_scaled, y_treino)

In [93]:
y_pred_v1 = modelo_logistico.predict(X_teste_scaled)
y_pred_v1[1:10]

array([0, 1, 1, 1, 1, 0, 1, 1, 0])

In [94]:
# Probabilidades das previsões
y_pred_prob_v1 = modelo_logistico.predict_proba(X_teste_scaled)[:,1]
y_pred_prob_v1[1:10]

array([0.00670437, 0.99529211, 0.87540493, 0.98140742, 0.86728891,
       0.2108784 , 0.97044303, 0.99042667, 0.08188889])

In [95]:
accuracy_v1 = accuracy_score(y_teste, y_pred_v1)
auc_v1 = roc_auc_score(y_teste, y_pred_prob_v1)
class_report_v1 = classification_report(y_teste, y_pred_v1)

print("Acurácia:", accuracy_v1)
print("AUC:", auc_v1)
print("Relatório de classificação:\n", class_report_v1)

Acurácia: 0.9166666666666666
AUC: 0.9788888888888889
Relatório de classificação:
               precision    recall  f1-score   support

           0       0.93      0.90      0.92        30
           1       0.90      0.93      0.92        30

    accuracy                           0.92        60
   macro avg       0.92      0.92      0.92        60
weighted avg       0.92      0.92      0.92        60



## Versão 2: Floresta Aleatória - Random Forest

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
modelo_randomforest = RandomForestClassifier()
modelo_randomforest.fit(X_treino_scaled, y_treino)

In [66]:
y_pred_v2 = modelo_randomforest.predict(X_teste_scaled)
y_pred_v2[1:10]

array([0, 1, 1, 1, 1, 0, 1, 1, 0])

In [67]:
y_pred_prob_v2 = modelo_randomforest.predict_proba(X_teste_scaled)
y_pred_prob_v2[1:10]

array([[1.  , 0.  ],
       [0.  , 1.  ],
       [0.1 , 0.9 ],
       [0.17, 0.83],
       [0.  , 1.  ],
       [0.55, 0.45],
       [0.01, 0.99],
       [0.01, 0.99],
       [0.92, 0.08]])

In [68]:
accuracy_v2 = accuracy_score(y_teste, y_pred_v2)
auc_v2 = roc_auc_score(y_teste, y_pred_prob_v2[:,1])
class_report_v2 = classification_report(y_teste, y_pred_v2)

print("Acurácia:", accuracy_v2)
print("AUC:", auc_v2)
print("Relatório de classificação:\n", class_report_v2)

Acurácia: 0.9166666666666666
AUC: 0.9572222222222223
Relatório de classificação:
               precision    recall  f1-score   support

           0       0.93      0.90      0.92        30
           1       0.90      0.93      0.92        30

    accuracy                           0.92        60
   macro avg       0.92      0.92      0.92        60
weighted avg       0.92      0.92      0.92        60



In [69]:
print("Acurácia:", accuracy_v1)
print("AUC:", auc_v1)
print("Relatório de classificação:\n", class_report_v1)

Acurácia: 0.9166666666666666
AUC: 0.9788888888888889
Relatório de classificação:
               precision    recall  f1-score   support

           0       0.93      0.90      0.92        30
           1       0.90      0.93      0.92        30

    accuracy                           0.92        60
   macro avg       0.92      0.92      0.92        60
weighted avg       0.92      0.92      0.92        60



## Versão 3: Support Vector Machines (SVM)

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [73]:
modelo_svm = SVC(probability=True)

In [ ]:
modelo_svm.fit(X_treino_scaled, y_treino)

In [81]:
y_pred_v3 = modelo_svm.predict(X_teste_scaled)
y_pred_prob_v3 = modelo_svm.predict_proba(X_teste_scaled)[:,1]

In [83]:
accuracy_v3 = accuracy_score(y_teste, y_pred_v3)
auc_v3 = roc_auc_score(y_teste, y_pred_prob_v3)
class_report_v3 = classification_report(y_teste, y_pred_v3)

print("Acurácia:", accuracy_v3)
print("AUC:", auc_v3)
print("Relatório de classificação:\n", class_report_v3)

Acurácia: 0.9333333333333333
AUC: 0.97
Relatório de classificação:
               precision    recall  f1-score   support

           0       0.93      0.93      0.93        30
           1       0.93      0.93      0.93        30

    accuracy                           0.93        60
   macro avg       0.93      0.93      0.93        60
weighted avg       0.93      0.93      0.93        60



## Escolha do modelo

Dado que os 3 modelos possuem acurácia próximas, e o primeiro modelo possui o melhor AUC e é mais simples que os demais, será escolhido o modelo 1: modelo Logístico.

In [97]:
print("Acurácia 1:", accuracy_v1)
print("Acurácia 2:", accuracy_v2)
print("Acurácia 3:", accuracy_v3)

Acurácia 1: 0.9166666666666666
Acurácia 2: 0.9166666666666666
Acurácia 3: 0.9333333333333333


In [96]:
print("AUC 1:", auc_v1)
print("AUC 2:", auc_v2)
print("AUC 3:", auc_v3)

AUC 1: 0.9788888888888889
AUC 2: 0.9572222222222223
AUC 3: 0.97


### Salvar modelo em disco


In [98]:
pickle.dump(modelo_logistico, open('modelo_final.pkl', 'wb'))
pickle.dump(scaler, open('scaler_final.pkl', 'wb'))